In [2]:
import pandas as pd
from datetime import datetime
#Loading csv 
df=pd.read_csv("C:\\Users\\KARTHICK\\OneDrive\\Desktop\\Python projects\\traffic_stops - traffic_stops_with_vehicle_number.csv")

C:\Users\KARTHICK\AppData\Local\Temp\ipykernel_11180\2912206040.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("C:\\Users\\KARTHICK\\OneDrive\\Desktop\\Python projects\\traffic_stops - traffic_stops_with_vehicle_number.csv")


In [2]:
df.head(5)

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305


In [7]:
# checking for dupliactes(row)
duplicates = df[df.duplicated()]
duplicates

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number


In [8]:
all_duplicates = df[df.duplicated(keep=False)]
all_duplicates

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number


In [9]:
#count of missing values in each column:
print(df.isna().sum())

stop_date                 0
stop_time                 0
country_name              0
driver_gender             0
driver_age_raw            0
driver_age                0
driver_race               0
violation_raw             0
violation                 0
search_conducted          0
search_type           21720
stop_outcome              0
is_arrested               0
stop_duration             0
drugs_related_stop        0
vehicle_number            0
dtype: int64


In [10]:
#as only search_type column only as null values,filling it as missing
df.fillna('data missing',inplace=True)

In [11]:
#dropping unwanted columns(data before cleaning):
df.drop(["driver_age_raw","violation_raw" ], axis=1, inplace=True)

In [6]:
print(df.isna().sum())

stop_date             0
stop_time             0
country_name          0
driver_gender         0
driver_age            0
driver_race           0
violation             0
search_conducted      0
search_type           0
stop_outcome          0
is_arrested           0
stop_duration         0
drugs_related_stop    0
vehicle_number        0
dtype: int64


In [21]:
#saving a copy of cleaned data:
df.to_csv('cleaned_data.csv', index=False)

In [22]:
df=pd.read_csv("cleaned_data.csv")

In [23]:
#changing date and time to timestamp:
df['time_stamp']=pd.to_datetime(df['stop_date']+' '+df['stop_time'], errors='coerce')
df['stop_date'] = pd.to_datetime(df['stop_date'], format='%Y-%m-%d', errors='coerce')
df['stop_time'] = pd.to_datetime(df['stop_time'], format='%H:%M:%S', errors='coerce').dt.time

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65538 entries, 0 to 65537
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   stop_date           65538 non-null  datetime64[ns]
 1   stop_time           65538 non-null  object        
 2   country_name        65538 non-null  object        
 3   driver_gender       65538 non-null  object        
 4   driver_age          65538 non-null  int64         
 5   driver_race         65538 non-null  object        
 6   violation           65538 non-null  object        
 7   search_conducted    65538 non-null  bool          
 8   search_type         65538 non-null  object        
 9   stop_outcome        65538 non-null  object        
 10  is_arrested         65538 non-null  bool          
 11  stop_duration       65538 non-null  object        
 12  drugs_related_stop  65538 non-null  bool          
 13  vehicle_number      65538 non-null  object    

In [11]:
#creating database named Policelogs:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

connection = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="Apple123",
    port = 5432
)

connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

writer = connection.cursor()

writer.execute("create database Policelogs") 

In [26]:
#pushing database to postgres using SQLalchemy:
from sqlalchemy import create_engine,TIMESTAMP, Date, Time, String, Boolean, Integer


host = "localhost"
port = "5432"  
database = "policelogs"
username = "postgres"
password = "Apple123"

engine_string = f"postgresql://{username}:{password}@{host}:{port}/{database}"

engine = create_engine(engine_string)

df = pd.read_csv("cleaned_data.csv")
dtype_map = {
    'stop_date': Date(),
    'stop_time': Time(),
    'country_name': String(),
    'driver_gender': String(),
    'driver_age': Integer(),
    'driver_race': String(),
    'violation': String(),
    'search_conducted': Boolean(),
    'search_type': String(),
    'stop_outcome': String(),
    'is_arrested': Boolean(),
    'stop_duration': String(),
    'drugs_related_stop': Boolean(),
    'vehicle_number': String(),
    'time_stamp': TIMESTAMP()
   
}

table_name = "traffic_logs"


df.to_sql(table_name, engine,if_exists='replace', index=False,dtype=dtype_map)  
print("Data successfully pushed to PostgreSQL table!")#for crosschecking

Data successfully pushed to PostgreSQL table!
